In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import sklearn
from scipy import spatial
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import cosine, correlation

In [ ]:
data = pd.read_csv("/content/u.data",sep = '\t')
data.columns =["userId", "itemId", "rating","timestamps"]
data

,userId,itemId,rating,timestamps
0,186,302,3,891717742
1,22,377,1,878887116
2,244,51,2,880606923
3,166,346,1,886397596
4,298,474,4,884182806
...,...,...,...,...
99994,880,476,3,880175444
99995,716,204,5,879795543
99996,276,1090,1,874795795
99997,13,225,2,882399156


In [ ]:
movie_matrix=np.zeros((1682,943))
total = len(data)
for i in range (total):
    u_id = data["userId"][i]-1
    m_id = data["itemId"][i]-1
    rating= data["rating"][i]
    movie_matrix[m_id][u_id]= rating

In [ ]:
movie_matrix.shape

(1682, 943)

In [ ]:
# SIMILARITY MATRIX
from sklearn.metrics.pairwise import cosine_similarity
similarity_matrix = cosine_similarity(movie_matrix)
similarity_matrix.shape

(1682, 1682)

In [ ]:
K=[10,20,30,40]
MAE=[]
for k in K:
    mae=0
    num_users=0
    for i in tqdm(range(len(data))):

        u_id = data["userId"][i]-1
        m_id = data["itemId"][i]-1
        actual_rating= data["rating"][i]

        predicted_rating=0
        total_sim=0
        similarity=[]
        rating=[]
        movie=[]

        for j in range(1682):
            if(j==m_id):
                continue
            if(movie_matrix[j][u_id]==0):
                continue
            similarity.append(similarity_matrix[j][m_id])
            rating.append(movie_matrix[j][u_id])

        top_k=sorted(range(len(similarity)), key=lambda i: similarity[i])[-k:]

        for p in top_k:
            predicted_rating+=rating[p]*similarity[p]
            total_sim += similarity[p]

        if total_sim != 0:
            #print(rating)
            mae += abs(actual_rating-(predicted_rating/total_sim))
            num_users+=1
        else:
            continue
    mae/=num_users
    MAE.append(mae)

100%|██████████| 99999/99999 [02:59<00:00, 557.45it/s]


In [ ]:
MAE

[0.7424832825218523,
 0.7473201514056342,
 0.7550281604162321,
 0.7614890736060862]